In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import *
import seaborn as sns
import missingno as mno
%matplotlib inline
#from autoimpute.imputations import SingleImputer
from fancyimpute import IterativeImputer, KNN
import itertools
from collections import Counter

from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import jaccard_similarity_score, f1_score, log_loss
from sklearn.metrics import classification_report, confusion_matrix

import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [2]:
def plot_loss_accuracy(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, max(1, historydf.values.max())))
    loss = history.history['loss'][-1]
    acc = history.history['acc'][-1]
    plt.title('Loss: %.3f, Accuracy: %.3f' % (loss, acc))

In [3]:
def plot_bin_loss_accuracy(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, max(1, historydf.values.max())))
    loss = history.history['loss'][-1]
    acc = history.history['binary_accuracy'][-1]
    plt.title('Loss: %.3f, Accuracy: %.3f' % (loss, acc))

In [4]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [5]:
train_data = pd.read_csv('train.csv')
print(train_data.shape)
train_data.head()

(164309, 14)


,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
1,10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
3,10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1


In [6]:
#train_data = train_data[(train_data.Months_Since_Deliquency < 83) | (train_data.Months_Since_Deliquency.isnull())]

In [7]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(train_data['Months_Since_Deliquency'].value_counts())

9.0      1518
10.0     1510
14.0     1487
12.0     1484
8.0      1474
         ... 
103.0       1
121.0       1
148.0       1
170.0       1
130.0       1
Name: Months_Since_Deliquency, Length: 122, dtype: int64


In [8]:
test_data = pd.read_csv('test.csv')
print(test_data.shape)
loan_id = test_data['Loan_ID']
test_data.head()

(109541, 13)


,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender
0,10164310,"27,500",10+ years,Mortgage,129000.0,VERIFIED - income,debt_consolidation,12.87,0,68.0,10,37,Male
1,10164311,"26,000",10+ years,NaN,110000.0,not verified,credit_card,11.37,0,NaN,6,23,Male
2,10164312,"6,075",< 1 year,Rent,75000.0,VERIFIED - income,debt_consolidation,6.83,2,NaN,5,20,Male
3,10164313,"12,000",10+ years,Mortgage,73000.0,VERIFIED - income source,debt_consolidation,7.76,0,NaN,6,8,Male
4,10164314,"35,000",< 1 year,Mortgage,156000.0,not verified,debt_consolidation,9.62,0,26.0,9,21,Male


In [9]:
train_data.columns = train_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
train_data = train_data.drop(['loan_id'], axis = 1)
train_data['months_since_deliquency'].fillna(0, inplace = True)
train_data["income_verified"].replace({"not verified": 'n', "VERIFIED - income": 'y', "VERIFIED - income source": 'y'}, inplace = True)
#train_data["gender"].replace({"Male": 0, "Female": 1}, inplace = True)
train_data['loan_amount_requested'] = train_data['loan_amount_requested'].apply(lambda x: x.replace(',',''))
train_data['loan_amount_requested'] = train_data['loan_amount_requested'].apply(pd.to_numeric, errors = 'coerce')
#train_data['length_employed'] = train_data.length_employed.str.extract('(\d+)')

test_data.columns = test_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
test_data = test_data.drop(['loan_id'], axis = 1)
test_data['months_since_deliquency'].fillna(0, inplace = True)
test_data["income_verified"].replace({"not verified": 'n', "VERIFIED - income": 'y', "VERIFIED - income source": 'y'}, inplace = True)
#test_data["gender"].replace({"Male": 0, "Female": 1}, inplace = True)
test_data['loan_amount_requested'] = test_data['loan_amount_requested'].apply(lambda x: x.replace(',',''))
test_data['loan_amount_requested'] = test_data['loan_amount_requested'].apply(pd.to_numeric, errors = 'coerce')
#test_data['length_employed'] = test_data.length_employed.str.extract('(\d+)')

train_data["length_employed"].fillna('NaN', inplace = True)
test_data["length_employed"].fillna('NaN', inplace = True)

train_data["annual_income"].fillna(train_data["annual_income"].mean(), inplace=True)
test_data["annual_income"].fillna(test_data["annual_income"].mean(), inplace=True)

train_data["home_owner"].fillna('NaN', inplace = True)
test_data["home_owner"].fillna('NaN', inplace = True)

print(train_data.shape)
train_data.isnull().sum()

(164309, 13)


loan_amount_requested      0
length_employed            0
home_owner                 0
annual_income              0
income_verified            0
purpose_of_loan            0
debt_to_income             0
inquiries_last_6mo         0
months_since_deliquency    0
number_open_accounts       0
total_accounts             0
gender                     0
interest_rate              0
dtype: int64

In [10]:
train_data.head()

,loan_amount_requested,length_employed,home_owner,annual_income,income_verified,purpose_of_loan,debt_to_income,inquiries_last_6mo,months_since_deliquency,number_open_accounts,total_accounts,gender,interest_rate
0,7000,< 1 year,Rent,68000.000000,n,car,18.37,0,0.0,9,14,Female,1
1,30000,4 years,Mortgage,73331.159434,y,debt_consolidation,14.93,0,17.0,12,24,Female,3
2,24725,7 years,Mortgage,75566.400000,y,debt_consolidation,15.88,0,0.0,12,16,Male,3
3,16000,< 1 year,NaN,56160.000000,y,debt_consolidation,14.34,3,0.0,16,22,Male,3
4,17000,8 years,Own,96000.000000,y,debt_consolidation,22.17,1,0.0,19,30,Female,1


In [14]:
# Defining X, y and splittinf the training data

X = train_data.drop('interest_rate', axis = 1).values
y = train_data['interest_rate'].values

# Normalize data
# Scaling the features within range (0, 1)

#X = preprocessing.StandardScaler().fit(X).transform(X)

#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 4, test_size = 0.33, shuffle = True)

In [15]:
kfold, scores = KFold(n_splits = 5, shuffle = True, random_state = 0), list()
for train, test in kfold.split(X):
    X_train, x_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    
    model = CatBoostClassifier(random_state = 27, max_depth = 6, n_estimators = 1000, verbose = 500)
    model.fit(X_train, y_train, cat_features=[1, 2, 4, 5, 11])
    preds = model.predict(x_test)
    score = f1_score(y_test, preds, average="weighted")
    scores.append(score)
    print(score)
print("Average: ", sum(scores)/len(scores))

Learning rate set to 0.10148
0:	learn: 1.0797082	total: 344ms	remaining: 5m 43s
500:	learn: 0.8985806	total: 2m 3s	remaining: 2m 3s
999:	learn: 0.8777947	total: 3m 55s	remaining: 0us
0.5328469144834992
Learning rate set to 0.10148
0:	learn: 1.0792872	total: 261ms	remaining: 4m 20s
500:	learn: 0.8972974	total: 1m 58s	remaining: 1m 57s
999:	learn: 0.8759680	total: 4m 6s	remaining: 0us
0.5305695453316372
Learning rate set to 0.10148
0:	learn: 1.0797655	total: 251ms	remaining: 4m 10s
500:	learn: 0.8994230	total: 2m 59s	remaining: 2m 59s
999:	learn: 0.8775709	total: 6m 29s	remaining: 0us
0.5311769943060832
Learning rate set to 0.10148
0:	learn: 1.0794087	total: 447ms	remaining: 7m 26s
500:	learn: 0.8985550	total: 3m 14s	remaining: 3m 14s
999:	learn: 0.8772975	total: 6m 24s	remaining: 0us
0.530915974086458
Learning rate set to 0.10148
0:	learn: 1.0796247	total: 431ms	remaining: 7m 10s
500:	learn: 0.8986963	total: 3m 6s	remaining: 3m 6s
999:	learn: 0.8766754	total: 6m 16s	remaining: 0us
0.532

In [16]:
X_train.shape

(131448, 12)

In [17]:
test_data.shape

(109541, 12)

In [18]:
test_preds = model.predict(test_data)

In [21]:
test_preds[0:5]

array([[2],
       [1],
       [2],
       [2],
       [2]], dtype=int64)

In [22]:
submission_df = pd.DataFrame({
                  "Loan_ID": loan_id, 
                  "Interest_Rate": test_preds.flatten()})

In [24]:
submission_df.to_csv('final_submission.csv', index = False)